* This document discusses automatically generating presentations from notebooks.  
* Presentations provide nonlinear narratives with x & y - slide & subslide - abstractions.  A nonlinear story is an intermediate to a linear story, or computational essay.
* The __reveal.js__ view of a notebook provides an alternative interface to view as slides and in a sorted layout view. 
* Headers have an added purpose to separate slides.

# Interpretting notebooks as presentations.

In [1]:
    import IPython; get_ipython = IPython.get_ipython
    ip = get_ipython()
    if ip:
        %reload_ext pidgin
    __all__ = 'present',

In [2]:
    import nbconvert, nbformat, yaml, io, traitlets

## Converting a notebook to a presentation.

The `nbconvert.SlidesExporter` exports `nbformat` documents as presentations;
`assert issubclass(nbconvert.SlidesExporter, nbconvert.HTMLExporter)`.

'    import nbconvert, nbformat, yaml, io, traitlets\n\n## Converting a notebook to a presentation.\n\nThe `nbconvert.SlidesExporter` exports `nbformat` documents as presentations;\n`assert issubclass(nbconvert.SlidesExporter, nbconvert.HTMLExporter)`.'

In [3]:
### Slide show metadata.

`nbconvert.SlidesExporter` uses cell metadata to create `"slide subslide fragment"` `"slide_type"`s.

    def add_slide_metadata(cell):
`add_slide_metadata` adds `"slideshow"` data to a __cell__ using the headers and horizontal rules as delimiters.

        source = ''.join(cell['source'])
        type = F"fragment slide slide subslide fragment fragment fragment".split(
        )[max(get_levels(source) or [0])]
`"slideshow"` is a keyword specific to the `nbconvert.SlidesExporter`, we only update the metadata 
if the keywords don't exist.

        cell.metadata['slideshow'] = cell.metadata.get('slideshow', {})
        cell.metadata['slideshow'].update(slide_type=cell.metadata['slideshow'].get(
            'slide_type', type
        ))
        return cell

'### Slide show metadata.\n\n`nbconvert.SlidesExporter` uses cell metadata to create `"slide subslide fragment"` `"slide_type"`s.\n\n    def add_slide_metadata(cell):\n`add_slide_metadata` adds `"slideshow"` data to a __cell__ using the headers and horizontal rules as delimiters.\n\n        source = \'\'.join(cell[\'source\'])\n        type = F"fragment slide slide subslide fragment fragment fragment".split(\n        )[max(get_levels(source) or [0])]\n`"slideshow"` is a keyword specific to the `nbconvert.SlidesExporter`, we only update the metadata \nif the keywords don\'t exist.\n\n        cell.metadata[\'slideshow\'] = cell.metadata.get(\'slideshow\', {})\n        cell.metadata[\'slideshow\'].update(slide_type=cell.metadata[\'slideshow\'].get(\n            \'slide_type\', type\n        ))\n        return cell'

In [4]:
### Extracting a table of contents with `mistune`

    import mistune, slugify
    class HeadingRenderer(mistune.Renderer):
        levels = None
        def header(self, text, level, raw=None):
            self.levels = (self.levels or set())
            self.levels.add(level)
            return text

'### Extracting a table of contents with `mistune`\n\n    import mistune, slugify\n    class HeadingRenderer(mistune.Renderer):\n        levels = None\n        def header(self, text, level, raw=None):\n            self.levels = (self.levels or set())\n            self.levels.add(level)\n            return text'

In [5]:
    def get_levels(text):
```
>>> assert get_levels("# asdf\\n## asdfadf") == {1, 2}
```

        renderer = HeadingRenderer()
        mistune.Markdown(renderer=renderer).render(text)

        return renderer.levels

'    def get_levels(text):\n```\n>>> assert get_levels("# asdf\\\\n## asdfadf") == {1, 2}\n```\n\n        renderer = HeadingRenderer()\n        mistune.Markdown(renderer=renderer).render(text)\n\n        return renderer.levels'

In [6]:
### Adding the metadata

Take image of the metadata.

    def add_slide_data(cell):
        source = ''.join(cell['source'])
        type = "fragment slide slide subslide fragment fragment fragment".split(
        )[max(get_levels(source) or [0])]
        cell.metadata['slideshow'] = cell.metadata.get('slideshow', {})
        cell.metadata['slideshow'].update(slide_type=cell.metadata['slideshow'].get('slide_type', type))
        return cell

'### Adding the metadata\n\nTake image of the metadata.\n\n    def add_slide_data(cell):\n        source = \'\'.join(cell[\'source\'])\n        type = "fragment slide slide subslide fragment fragment fragment".split(\n        )[max(get_levels(source) or [0])]\n        cell.metadata[\'slideshow\'] = cell.metadata.get(\'slideshow\', {})\n        cell.metadata[\'slideshow\'].update(slide_type=cell.metadata[\'slideshow\'].get(\'slide_type\', type))\n        return cell'

In [7]:
    class AutomaticallyPresent(nbconvert.preprocessors.Preprocessor):
### An `nbconvert.preprocessors.Preprocessor`
    
    

        def preprocess_cell(self, cell, resources, index):
            if 'source' not in cell: return cell, resources
            
            if cell['cell_type'] == 'code':
                self.is_pidgin = self.is_pidgin or (
                    'load_ext pidgin' in ''.join(cell['source']))
               
            if cell['cell_type'] in (['markdown'] + (
                self.is_pidgin and ['code'] or []
            )):
                if (cell['cell_type'] == 'markdown') or ('outputs' in cell) and cell['outputs']:
                    cell = add_slide_data(cell)
                
            return cell, resources
        
        def preprocess(self, nb, resources):
            self.last_heading = None
            self.is_pidgin = False
            while nb.cells and (
                not (nb.cells[-1].source or nb.cells[-1].outputs)
            ): nb.cells.pop()
            if nb.cells and nb.cells[-1]['cell_type'] == 'raw':
                nb.metadata.merge(yaml.safe_load(io.StringIO(''.join(nb.cells.pop()['source']))))
                
            nb, resources = super().preprocess(nb, resources)
            return nbconvert.exporters.slides.prepare(nb), resources


"    class AutomaticallyPresent(nbconvert.preprocessors.Preprocessor):\n### An `nbconvert.preprocessors.Preprocessor`\n    \n    \n\n        def preprocess_cell(self, cell, resources, index):\n            if 'source' not in cell: return cell, resources\n            \n            if cell['cell_type'] == 'code':\n                self.is_pidgin = self.is_pidgin or (\n                    'load_ext pidgin' in ''.join(cell['source']))\n               \n            if cell['cell_type'] in (['markdown'] + (\n                self.is_pidgin and ['code'] or []\n            )):\n                if (cell['cell_type'] == 'markdown') or ('outputs' in cell) and cell['outputs']:\n                    cell = add_slide_data(cell)\n                \n            return cell, resources\n        \n        def preprocess(self, nb, resources):\n            self.last_heading = None\n            self.is_pidgin = False\n            while nb.cells and (\n                not (nb.cells[-1].source or nb.cells[-1].outputs)\n            ): nb.cells.pop()\n            if nb.cells and nb.cells[-1]['cell_type'] == 'raw':\n                nb.metadata.merge(yaml.safe_load(io.StringIO(''.join(nb.cells.pop()['source']))))\n                \n            nb, resources = super().preprocess(nb, resources)\n            return nbconvert.exporters.slides.prepare(nb), resources\n"

## Automatically adding the metadata.

In [8]:
    mod_css = lambda str: str.replace("""                .css('height', 'calc(95vh)')
                    .css('overflow-y', 'scroll')
                    .css('margin-top', '20px');""",
    """                .css('height', 'calc(100vh)')
                .css('overflow-y', 'scroll')
                .css('margin-top', '0px');""")

In [9]:
    def present(filename, **dict):
        import vdom.svg
        exporter = nbconvert.SlidesExporter(
            preprocessors=[AutomaticallyPresent()],
            reveal_scroll=True, exclude_output_prompt=True, exclude_raw=True,
            exclude_input=True, **dict
        )
        template = exporter.environment.loader.loaders[0].get_source(exporter.environment, 'slides_reveal')[0]
        lead, sep, template = template.partition('<section')
        sep += """ {% for key, value in nb.metadata.get('dataset', {}).items()%}data-{{key}}="{{value}}" {% endfor %}"""
        template = template.replace("""<section""", """<section""" + """ {% for key, value in cell.metadata.get('dataset', {}).items()%}data-{{key}}="{{value}}" {% endfor %}""")
        template = lead + sep + template
        template = template.replace('controls: true,', '''width: "100%", height: "100%", margin: 0, controls: true, slideNumber: true, center: false, loop: true, backgroundTransition: 'slide', width: '100%', ''')
        template = mod_css(template)
        template = template.replace('</body>', '</body><style>li code, p code {display: contents;}</style>')
        template = template.replace(
            """Reveal.addEventListener('slidechanged', setScrollingSlide);""",
            """Reveal.addEventListener('slidechanged', setScrollingSlide);
            $('.slide-number').click(function(){Reveal.toggleOverview();});

            """)
        exporter._template_cached = exporter.environment.from_string(template)
        return vdom.svg.iframe(
            srcdoc=exporter.from_filename(filename)[0],
            width="100%", height="500px",
            **{"class": "pdg-Frame"}
        )._repr_html_()